Import

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras import Model, models, layers, optimizers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint

seed=1
tf.random.set_seed(seed)

# Make dataset for train

In [2]:
# read_csv

path = "../dataset/data/Media/"
data1 = pd.read_csv(path + "Media_INFO.csv", index_col="Timestamp", )
data2 = pd.read_csv(path + "Media_LOGIN.csv", index_col="Timestamp")
data3 = pd.read_csv(path + "Media_MENU.csv", index_col="Timestamp")
data4 = pd.read_csv(path + "Media_STREAM.csv", index_col="Timestamp")

data1.reset_index(inplace=True, drop=True)
data2.reset_index(inplace=True, drop=True)
data3.reset_index(inplace=True, drop=True)
data4.reset_index(inplace=True, drop=True)

In [3]:
# drop na, column

data1.drop('INFO-01-Fail', axis=1, inplace=True)
data1.dropna(inplace=True)
data2.dropna(inplace=True)
data3.dropna(inplace=True)
data4.dropna(inplace=True)

server1 = data1
server2 = data2.iloc[:, [0, 1, 2]]
server3 = data2.iloc[:, [3, 4, 5]]
server4 = data2.iloc[:, [6, 7, 8]]
server5 = data2.iloc[:, [9, 10, 11]]
server6 = data2.iloc[:, [12, 13, 14]]
server7 = data3.iloc[:, [0, 1, 2]]
server8 = data3.iloc[:, [3, 4, 5]]
server9 = data3.iloc[:, [6, 7, 8]]
server10 = data3.iloc[:, [9, 10, 11]]
server11 = data4.iloc[:, [0]]
server12 = data4.iloc[:, [1]]
server13 = data4.iloc[:, [2]]

In [4]:
for i in range(13):
    print(globals()[f'server{i + 1}'].shape)

(192249, 2)
(192189, 3)
(192189, 3)
(192189, 3)
(192189, 3)
(192189, 3)
(209771, 3)
(209771, 3)
(209771, 3)
(209771, 3)
(178730, 1)
(178730, 1)
(178730, 1)


In [5]:
# remove outlier 
def remove_outlier(df, column):
    Q1 = np.percentile(df[column].values, 25)
    Q3 = np.percentile(df[column].values, 75)
    
    IQR = Q3 - Q1
    weight = IQR*1.5
    
    lowest = Q1 - weight
    highest = Q3 + weight
    
    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
    df.drop(outlier_idx, axis=0, inplace=True)
    
    return df

In [6]:
for i in range(13):
    column_list = list(globals()[f'server{i + 1}'].columns)
    for j in column_list:
        remove_outlier(globals()[f'server{i + 1}'], j)

/var/folders/qd/yc_92c9d2csclp39hh5l_t900000gn/T/ipykernel_20422/2360512852.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(outlier_idx, axis=0, inplace=True)


In [7]:
for i in range(13):
    print(globals()[f'server{i + 1}'].shape)

(188513, 2)
(170651, 3)
(170739, 3)
(170815, 3)
(170750, 3)
(170718, 3)
(198213, 3)
(198233, 3)
(198221, 3)
(198162, 3)
(178534, 1)
(178730, 1)
(173192, 1)


In [8]:
# df add

server_df = []
for i in range(13):
    server_df.append(globals()[f'server{i + 1}'].reset_index(drop=True))

# Transform to sequence

In [9]:
def to_seq(df, step):
    output = []
    for i in range(len(df) - step - 1):
        temp = []
        for j in range(1, step + 1):
            temp.append(df.loc[[(i + j + 1)], :])
        output.append(temp)
    return np.squeeze(np.array(output))

In [10]:
step = abs(5)
trans_server_df = []
for i in range(len(server_df)):
    temp = (to_seq(server_df[i], step))
    trans_server_df.append(temp)
    print(temp.shape)

(188507, 5, 2)
(170645, 5, 3)
(170733, 5, 3)
(170809, 5, 3)
(170744, 5, 3)
(170712, 5, 3)
(198207, 5, 3)
(198227, 5, 3)
(198215, 5, 3)
(198156, 5, 3)
(178528, 5)
(178724, 5)
(173186, 5)


# Split Valid

In [11]:
train = []
valid = []
for i in range(len(trans_server_df)):
    temp1, temp2 = train_test_split(trans_server_df[i], test_size=0.2)
    train.append(temp1)
    valid.append(temp2)


for i in range(len(train)):
    print(len(train[i]))
    print(len(valid[i]))

150805
37702
136516
34129
136586
34147
136647
34162
136595
34149
136569
34143
158565
39642
158581
39646
158572
39643
158524
39632
142822
35706
142979
35745
138548
34638


# feature scale

In [12]:
def dimension_down(df):
    dimension_df = np.empty((df.shape[0], df.shape[2]))
    for i in range(df.shape[0]):
        dimension_df[i] = df[i, (df.shape[1] - 1), :]

    return dimension_df


def scaling(df, scaler):
    for i in range(df.shape[0]):
        df[i, :, :] = scaler.transform(df[i, :, :])

    return df

In [13]:
train_scaled = []
valid_scaled = []
for i in range(10):
    scaler = RobustScaler().fit(dimension_down(train[i]))
    temp1 = scaling(train[i], scaler)
    temp2 = scaling(valid[i], scaler)
    train_scaled.append(temp1)
    valid_scaled.append(temp2)
    
for j in range(3):
    scaler = RobustScaler().fit(train[j + 10])
    temp1 = scaler.transform(train[j + 10])
    temp2 = scaler.transform(valid[j + 10])
    train_scaled.append(temp1)
    valid_scaled.append(temp2)
    
test_scaled = train_scaled.copy()

# LSTM AE for trainset

In [14]:
feature_number = []
for i in range(len(train_scaled) - 3):
    feature_number.append(train_scaled[i].shape[2])
    
for j in range(3):
    feature_number.append(1)

In [15]:
def lstm_autoencoder(i):   
    lstm_ae = models.Sequential()

    # Encoding
    lstm_ae.add(layers.LSTM(64, activation='tanh', input_shape=(step, feature_number[i]), return_sequences=True))
    lstm_ae.add(layers.LSTM(32, activation='tanh', return_sequences=False))
    lstm_ae.add(layers.RepeatVector(step))

    # Decoding
    lstm_ae.add(layers.LSTM(32, activation='tanh', return_sequences=True))
    lstm_ae.add(layers.LSTM(64, activation='tanh', return_sequences=True))
    lstm_ae.add(layers.TimeDistributed(layers.Dense(feature_number[i])))

    # earlystopping
    callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)

    print(lstm_ae.summary())
    return lstm_ae

In [16]:
for i in range(13):
    globals()[f'lstm_ae{i + 1}'] = lstm_autoencoder(i)

2022-05-09 20:17:56.743187: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-09 20:17:56.745476: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 64)             17152     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 repeat_vector (RepeatVector  (None, 5, 32)            0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 5, 32)             8320      
                                                                 
 lstm_3 (LSTM)               (None, 5, 64)             24832     
                                                                 
 time_distributed (TimeDistr  (None, 5, 2)    

In [17]:
epochs = 10
batch_size = 128
learning_rate = 0.001

In [18]:
# compile

for i in range(13):
    globals()[f"lstm_ae{i + 1}"].compile(
        loss="mse", optimizer=optimizers.Adam(learning_rate)
    )

In [19]:
# fit

for i in range(13):
    history = globals()[f"lstm_ae{i + 1}"].fit(
        train_scaled[i],
        train_scaled[i],
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(valid_scaled[i], valid_scaled[i]),
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)],
    )

Epoch 1/10


2022-05-09 20:18:02.107414: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-09 20:18:03.744424: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-09 20:18:04.180492: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-09 20:18:04.258169: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-09 20:18:04.305635: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-09 20:18:04.338657: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-09 20:18:04.438293: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113

   5/1179 [..............................] - ETA: 30s - loss: 0.4582

2022-05-09 20:18:04.680226: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1179/1179 [==============================] - ETA: 0s - loss: 0.0279

...

2022-05-09 20:52:47.521826: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1239/1239 [==============================] - ETA: 0s - loss: 0.0460

2022-05-09 20:53:12.164936: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-09 20:53:12.317509: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-09 20:53:12.370023: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-09 20:53:12.444869: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-09 20:53:12.485845: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1239/1239 [==============================] - 31s 22ms/step - loss: 0.0460 - val_loss: 0.0251
Epoch 2/10
1239/1239 [==============================] - 26s 21ms/step - loss: 0.0183 - val_loss: 0.0133
Epoch 3/10
1239/1239 [==============================] - 26s 21ms/step - loss: 0.0098 - val_loss: 0.0044
Epoch 4/10
1239/1239 [==============================] - 26s 21ms/step - loss: 0.0038 - val_loss: 0.0034
Epoch 5/10
1239/1239 [==============================] - 26s 21ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 6/10
1239/1239 [==============================] - 26s 21ms/step - loss: 0.0023 - val_loss: 0.0020
Epoch 7/10
1239/1239 [==============================] - 27s 21ms/step - loss: 0.0017 - val_loss: 0.0010
Epoch 8/10
1124/1239 [==========================>...] - ETA: 2s - loss: 5.4236e-04

KeyboardInterrupt: 

# Find normal

In [ ]:
predictions = []
for i in range(13):
    temp = globals()[f"lstm_ae{i + 1}"].predict(test_scaled[i])
    predictions.append(temp)

In [ ]:
threshold = 0.1

In [ ]:
predict = []
for j in range(13):
    for i in range(len(test_scaled[j])):
        temp = []
        if np.sqrt(np.mean(np.power(predictions[j].reshape(predictions[j].shape[0], predictions[j].shape[1] * predictions[j].shape[2])[i] - test_scaled[j][0].reshape(predictions[j].shape[0], predictions[j].shape[1] * predictions[j].shape[2])[i], 2))) > threshold:
            temp.append(1)
        else:
            temp.append(0)
        predict.append(temp)

In [ ]:
for i in range(13):
    print(pd.DataFrame(predict[i]).value_counts())